In [101]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import yfinance as yf

# Functions

In [102]:
def format_columns(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    df.date = pd.to_datetime(df.date)
    return df

def round_values(df):
    for column in ['open', 'high', 'low', 'close', 'adj_close', 'volume']:
        df[column] = df[column].apply(lambda x: round(x, 3))
    return df

def analysis_df(df):
    df['daily_returns'] = df.adj_close.pct_change() * 100
    df['daily_returns'] = df['daily_returns'].bfill()
    df['daily_range'] = df.high - df.low

    summary = {}
    
    for column in ['open', 'high', 'low', 'close', 'adj_close', 'volume', 'daily_returns', 'daily_range']:
        column_mean = round(df[column].mean(), 3)
        column_std = round(df[column].std(), 3)
        summary[column] = [column_mean, column_std]
    
    summary['monthly_returns'] = [round(monthly_and_annual_returns_stats(df)[0], 3), round(monthly_and_annual_returns_stats(df)[1], 3)]
    summary['annual_returns'] = [round(monthly_and_annual_returns_stats(df)[2], 3), round(monthly_and_annual_returns_stats(df)[3], 3)]

    df = pd.DataFrame(summary, index=['mean', 'std'])
    
    return df

def monthly_and_annual_returns_stats(df):
    
    # Calculate monthly returns using the last available 'adj_close' within each month
    monthly_returns = df.set_index('date').resample("ME")['adj_close'].last().pct_change() * 100
    annual_returns = df.set_index('date').resample("YE")['adj_close'].last().pct_change() * 100
    print(len(monthly_returns))
    print(len(annual_returns))
    mean_monthly_returns = monthly_returns.mean()
    std_monthly_returns = monthly_returns.std()
    mean_annual_returns = annual_returns.mean()
    std_annual_returns = annual_returns.std()

    return mean_monthly_returns, std_monthly_returns, mean_annual_returns, std_annual_returns

def adj_close_analysis(df, company):
    plt.figure(figsize=(9,3))
    plt.plot(df.date ,df.adj_close, c = 'red')
    plt.title('Adjustment Closing Value of ' + company)
    plt.ylabel('Stock Price')
    plt.show()

def volume_analysis(df, company):
    plt.figure(figsize=(9,3))
    plt.plot(df.date ,df.volume, c = 'blue')
    plt.title('Trading Volume of ' + company)
    plt.ylabel('Volume')
    plt.show()

# JP Morgan

In [103]:
df_JPM_2000 = pd.read_csv('../data/raw/JPM.csv')
df_JPM_2000 = format_columns(df_JPM_2000)
df_JPM_2000 = round_values(df_JPM_2000)

df_JPM_2024 = pd.read_csv('../data/raw/JPM2024.csv')
df_JPM_2024 = format_columns(df_JPM_2024)
df_JPM_2024 = round_values(df_JPM_2024)

In [104]:
df_JPM = pd.concat([df_JPM_2000, df_JPM_2024], axis=0)
df_JPM = df_JPM.drop_duplicates(subset='date', keep='first')
df_JPM = df_JPM[df_JPM.date >= '2019-11-01']
df_JPM = df_JPM.reset_index(drop=True)

In [105]:
df_JPM = yf.download('JPM', start='2019-11-01')
df_JPM.columns = df_JPM.columns.droplevel(1)
df_JPM.columns = df_JPM.columns.str.strip().str.lower().str.replace(' ', '_')
df_JPM['date'] = df_JPM.index.strftime("%Y-%m-%d")
df_JPM['date'] = pd.to_datetime(df_JPM['date'])
df_JPM.head()

[*********************100%***********************]  1 of 1 completed


Price,adj_close,close,high,low,open,volume,date
Date,,,,,,,
2019-11-01 00:00:00+00:00,110.602936,127.800003,127.989998,126.019997,126.199997,11682600,2019-11-01
2019-11-04 00:00:00+00:00,111.485703,128.820007,129.419998,128.559998,129.080002,9159900,2019-11-04
2019-11-05 00:00:00+00:00,111.728020,129.100006,130.300003,128.839996,129.389999,13747600,2019-11-05
2019-11-06 00:00:00+00:00,111.901115,129.300003,129.839996,128.580002,128.690002,11231100,2019-11-06
2019-11-07 00:00:00+00:00,112.506912,130.000000,131.289993,129.919998,130.429993,13528900,2019-11-07


In [106]:
#adj_close_analysis(df_JPM, 'JP Morgan')

In [107]:
#volume_analysis(df_JPM, 'JP Morgan')

In [108]:
analysis_df(df_JPM)
df_JPM.head(3)

61
6
61
6
61
6
61
6


Price,adj_close,close,high,low,open,volume,date,daily_returns,daily_range
Date,,,,,,,,,
2019-11-01 00:00:00+00:00,110.602936,127.800003,127.989998,126.019997,126.199997,11682600,2019-11-01,0.798140,1.970001
2019-11-04 00:00:00+00:00,111.485703,128.820007,129.419998,128.559998,129.080002,9159900,2019-11-04,0.798140,0.860001
2019-11-05 00:00:00+00:00,111.728020,129.100006,130.300003,128.839996,129.389999,13747600,2019-11-05,0.217353,1.460007


# Goldman Sachs

In [109]:
df_GS_2000 = pd.read_csv('../data/raw/GoldmanSachs.csv')
df_GS_2000 = format_columns(df_GS_2000)
df_GS_2000 = round_values(df_GS_2000)

df_GS_2024 = pd.read_csv('../data/raw/GoldmanSachs2024.csv')
df_GS_2024 = format_columns(df_GS_2024)
df_GS_2024 = round_values(df_GS_2024)

In [110]:
df_GS = pd.concat([df_GS_2000, df_GS_2024], axis=0)
df_GS = df_GS.drop_duplicates(subset='date', keep='first')
df_GS = df_GS[df_GS.date >= '2019-11-01']
df_GS = df_GS.reset_index(drop=True)
df_GS.head()

,date,open,high,low,close,adj_close,volume
0,2019-11-01,215.26,217.89,214.75,217.39,200.676,2112300
1,2019-11-04,219.70,220.69,218.57,219.87,202.966,2538100
2,2019-11-05,220.80,221.78,218.47,218.64,201.830,2525100
3,2019-11-06,218.01,219.07,216.55,218.42,201.627,1863000
4,2019-11-07,220.77,224.77,220.41,223.29,206.123,3748600


In [111]:
df_GS = yf.download('GS', start='2019-11-01')
df_GS.columns = df_GS.columns.droplevel(1)
df_GS.columns = df_GS.columns.str.strip().str.lower().str.replace(' ', '_')
df_GS['date'] = df_GS.index.strftime("%Y-%m-%d")
df_GS['date'] = pd.to_datetime(df_GS['date'])
df_GS.head()

[*********************100%***********************]  1 of 1 completed


Price,adj_close,close,high,low,open,volume,date
Date,,,,,,,
2019-11-01 00:00:00+00:00,192.218445,217.389999,217.889999,214.750000,215.259995,2112300,2019-11-01
2019-11-04 00:00:00+00:00,194.411316,219.869995,220.690002,218.570007,219.699997,2538100,2019-11-04
2019-11-05 00:00:00+00:00,193.323700,218.639999,221.779999,218.470001,220.800003,2525100,2019-11-05
2019-11-06 00:00:00+00:00,193.129166,218.419998,219.070007,216.550003,218.009995,1863000,2019-11-06
2019-11-07 00:00:00+00:00,197.435272,223.289993,224.770004,220.410004,220.770004,3748600,2019-11-07


In [112]:
#adj_close_analysis(df_GS, 'Goldman Sachs')

In [113]:
#volume_analysis(df_GS, 'Goldman Sachs')

In [114]:
analysis_df(df_GS)
df_GS.head(3)

61
6
61
6
61
6
61
6


Price,adj_close,close,high,low,open,volume,date,daily_returns,daily_range
Date,,,,,,,,,
2019-11-01 00:00:00+00:00,192.218445,217.389999,217.889999,214.750000,215.259995,2112300,2019-11-01,1.140822,3.139999
2019-11-04 00:00:00+00:00,194.411316,219.869995,220.690002,218.570007,219.699997,2538100,2019-11-04,1.140822,2.119995
2019-11-05 00:00:00+00:00,193.323700,218.639999,221.779999,218.470001,220.800003,2525100,2019-11-05,-0.559441,3.309998


# Microsoft

In [115]:
df_microsoft = yf.download('NVDA', start='2019-11-01')
df_microsoft.columns = df_microsoft.columns.droplevel(1)
df_microsoft.columns = df_microsoft.columns.str.strip().str.lower().str.replace(' ', '_')
df_microsoft['date'] = df_microsoft.index.strftime("%Y-%m-%d")
df_microsoft['date'] = pd.to_datetime(df_microsoft['date'])
df_microsoft.head()

[*********************100%***********************]  1 of 1 completed


Price,adj_close,close,high,low,open,volume,date
Date,,,,,,,
2019-11-01 00:00:00+00:00,5.040235,5.06475,5.09725,4.96475,4.99000,283640000,2019-11-01
2019-11-04 00:00:00+00:00,5.237028,5.26250,5.27500,5.12300,5.14625,376716000,2019-11-04
2019-11-05 00:00:00+00:00,5.214885,5.24025,5.28850,5.19200,5.26225,300868000,2019-11-05
2019-11-06 00:00:00+00:00,5.165625,5.19075,5.24175,5.10275,5.21600,269012000,2019-11-06
2019-11-07 00:00:00+00:00,5.183289,5.20850,5.29250,5.16350,5.27775,280340000,2019-11-07


In [116]:
df_nvidia = yf.download('NVDA', start='2019-11-01')
df_nvidia.columns = df_nvidia.columns.droplevel(1)
df_nvidia.columns = df_nvidia.columns.str.strip().str.lower().str.replace(' ', '_')
df_nvidia['date'] = df_nvidia.index.strftime("%Y-%m-%d")
df_nvidia['date'] = pd.to_datetime(df_nvidia['date'])
df_nvidia.head()

[*********************100%***********************]  1 of 1 completed


Price,adj_close,close,high,low,open,volume,date
Date,,,,,,,
2019-11-01 00:00:00+00:00,5.040235,5.06475,5.09725,4.96475,4.99000,283640000,2019-11-01
2019-11-04 00:00:00+00:00,5.237028,5.26250,5.27500,5.12300,5.14625,376716000,2019-11-04
2019-11-05 00:00:00+00:00,5.214885,5.24025,5.28850,5.19200,5.26225,300868000,2019-11-05
2019-11-06 00:00:00+00:00,5.165625,5.19075,5.24175,5.10275,5.21600,269012000,2019-11-06
2019-11-07 00:00:00+00:00,5.183289,5.20850,5.29250,5.16350,5.27775,280340000,2019-11-07


# Comparison

In [117]:
analysis_JPM = analysis_df(df_JPM)
analysis_JPM.to_csv('../data/clean/analysis_JPM.csv', index=False)
analysis_JPM

61
6
61
6
61
6
61
6


,open,high,low,close,adj_close,volume,daily_returns,daily_range,monthly_returns,annual_returns
mean,144.594,146.040,143.174,144.641,135.889,1.313218e+07,0.083,2.867,1.578,17.228
std,32.239,32.311,32.224,32.319,34.810,7.039448e+06,2.030,1.483,7.945,25.125


In [118]:
analysis_GS = analysis_df(df_GS)
analysis_GS.to_csv('../data/clean/analysis_GS.csv', index=False)
analysis_GS

61
6
61
6


61
6
61
6


,open,high,low,close,adj_close,volume,daily_returns,daily_range,monthly_returns,annual_returns
mean,330.859,334.589,327.286,331.005,312.909,2694719.265,0.111,7.302,2.245,25.833
std,84.649,85.214,84.143,84.677,88.433,1362002.106,2.074,3.526,9.006,25.947


In [119]:
analysis_microsoft = analysis_df(df_microsoft)
analysis_microsoft.to_csv('../data/clean/analysis_MSFT.csv', index=False)
analysis_microsoft

61
6
61
6
61
6
61
6


,open,high,low,close,adj_close,volume,daily_returns,daily_range,monthly_returns,annual_returns
mean,34.415,35.063,33.720,34.425,34.401,4.500752e+08,0.326,1.343,6.615,125.479
std,34.278,34.902,33.548,34.251,34.257,1.861858e+08,3.381,1.641,14.346,109.615


In [120]:
analysis_nvidia = analysis_df(df_nvidia)
analysis_nvidia.to_csv('../data/clean/analysis_NVDA.csv', index=False)
analysis_nvidia

61
6
61
6
61
6
61
6


,open,high,low,close,adj_close,volume,daily_returns,daily_range,monthly_returns,annual_returns
mean,34.415,35.063,33.720,34.425,34.401,4.500752e+08,0.326,1.343,6.615,125.479
std,34.278,34.902,33.548,34.251,34.257,1.861858e+08,3.381,1.641,14.346,109.615
